# Merging Duplicated Subjects in PNC

We want congruent naming between both Flywheel projects of PNC. Instead of just renaming the BIDS namespace data, we'll also rename and merge subjects who are zero padded.

TL;DR -- In this jupyter session, we:

1. Gathered subjects who had zero padded duplicates
2. Copied their session and acquisition data to their non-zero padded entities
3. Wrote a function to transfer files between two entities by downlaoding to disk and uploading to destination
4. Realised this function failed somewhere, and had NIFTIs that didn't have metadata copied over
5. Deleted all the niftis from these new sessions and ran dcm2niix to produce clean new niftis

In [1]:
import flywheel

In [2]:
fw = flywheel.Client()

In [3]:
lg = fw.projects.find_first('label="PNC_LG_810336"')
cs = fw.projects.find_first('label="PNC_CS_810336"')

/Users/ttapera/miniconda3/envs/fw_bids/lib/python3.7/site-packages/flywheel/flywheel.py:5757: UserWarning: Client version 8.6.0 does not match server version 9.0.0-beta.2. Please update your client version!
  warnings.warn('Client version {} does not match server version {}. Please update your client version!'.format(SDK_VERSION, release_version))


In [4]:
subs_cs = cs.subjects()
subs_lg = lg.subjects()

In [5]:
len([int(x.label) for x in subs_cs])

1605

In [6]:
len(set([int(x.label) for x in subs_cs]))

1605

In [7]:
len([int(x.label) for x in subs_lg])

702

In [8]:
len(set([int(x.label) for x in subs_lg]))

619

In LG, if we parse all of the subject labels as ints, and find the unique values in the list of subjects, the list shrinks. This means participants in LG have zero padding and have been duplicated, eg. sub-01 and sub-1 are the same subject.

We can copy all of the data from the zero-padded duplicate and paste it into the non-padded original.

Below we print the index and subject ID of these duplicates:

In [9]:
sublabels = [x.label for x in subs_lg]
dupes = {}

for s in subs_lg:
    
    if str(int(s.label)) != s.label and str(int(s.label)) in sublabels:
        dupes[s.label] = str(int(s.label))

In [10]:
dupes

{'080263': '80263',
 '080557': '80557',
 '081222': '81222',
 '081323': '81323',
 '081462': '81462',
 '081939': '81939',
 '082051': '82051',
 '082063': '82063',
 '082066': '82066',
 '083010': '83010',
 '083454': '83454',
 '083835': '83835',
 '083987': '83987',
 '084103': '84103',
 '085063': '85063',
 '085083': '85083',
 '085369': '85369',
 '085400': '85400',
 '085412': '85412',
 '086626': '86626',
 '086946': '86946',
 '087135': '87135',
 '087144': '87144',
 '087166': '87166',
 '087277': '87277',
 '087804': '87804',
 '087990': '87990',
 '088181': '88181',
 '088402': '88402',
 '088589': '88589',
 '088760': '88760',
 '088770': '88770',
 '088773': '88773',
 '088867': '88867',
 '088902': '88902',
 '089115': '89115',
 '089377': '89377',
 '090077': '90077',
 '090113': '90113',
 '090281': '90281',
 '090369': '90369',
 '090613': '90613',
 '090877': '90877',
 '091025': '91025',
 '091335': '91335',
 '091341': '91341',
 '091726': '91726',
 '091855': '91855',
 '091919': '91919',
 '091962': '91962',


In [11]:
true_example = [x for x in subs_lg if x.label == '80263'][0]
dupe_ex = [x for x in subs_lg if x.label == '080263'][0]

In [13]:
ex_sessions = true_example.sessions()
ex_sessions

[{'age': None,
  'analyses': None,
  'created': datetime.datetime(2019, 2, 27, 22, 49, 54, 519000, tzinfo=tzutc()),
  'files': [{'classification': {},
             'created': datetime.datetime(2019, 2, 27, 22, 56, 28, 103000, tzinfo=tzutc()),
             'hash': '',
             'id': '7aff1f1c-d3e8-47c9-8cd6-908ed56eb48e',
             'info': {},
             'info_exists': False,
             'mimetype': 'text/xml',
             'modality': None,
             'modified': datetime.datetime(2019, 2, 28, 17, 43, 44, 501000, tzinfo=tzutc()),
             'name': '009127_20141113_125621.xml',
             'origin': {'id': 'harshakethineni@flywheel.io',
                        'method': None,
                        'name': None,
                        'type': 'user',
                        'via': None},
             'replaced': datetime.datetime(2019, 2, 28, 17, 43, 44, 814000, tzinfo=tzutc()),
             'size': 18741,
             'tags': [],
             'type': 'markup',
       

In [14]:
dupe_sessions = dupe_ex.sessions()
dupe_sessions

[{'age': None,
  'analyses': None,
  'created': datetime.datetime(2019, 3, 1, 16, 16, 42, 825000, tzinfo=tzutc()),
  'files': [{'classification': {},
             'created': datetime.datetime(2019, 3, 1, 16, 53, 2, 315000, tzinfo=tzutc()),
             'hash': '',
             'id': '3bce355f-418f-495b-9624-f76e8ac58409',
             'info': {},
             'info_exists': False,
             'mimetype': 'application/octet-stream',
             'modality': None,
             'modified': datetime.datetime(2019, 3, 1, 16, 53, 1, 923000, tzinfo=tzutc()),
             'name': 'dcmtoxnat.log',
             'origin': {'id': 'harshakethineni@flywheel.io',
                        'method': None,
                        'name': None,
                        'type': 'user',
                        'via': None},
             'replaced': None,
             'size': 10251,
             'tags': [],
             'type': 'log',
             'zip_member_count': None},
            {'classification': {},

In [16]:
#true_example.add_session(label=dupe_sessions[0].label)

{'age': None,
 'analyses': [],
 'created': datetime.datetime(2019, 8, 7, 20, 19, 4, 285000, tzinfo=tzutc()),
 'files': [],
 'group': 'bbl',
 'id': '5d4b323ac3758a0047d862ca',
 'info': {},
 'info_exists': None,
 'label': '008454',
 'modified': datetime.datetime(2019, 8, 7, 20, 19, 4, 285000, tzinfo=tzutc()),
 'notes': [],
 'operator': None,
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'bbl',
             'project': '5c76eee2ba2580003839f897',
             'session': None,
             'subject': '5c7713ccba2580002738df72'},
 'permissions': [{'access': 'admin', 'id': 'harshakethineni@flywheel.io'},
                 {'access': 'admin', 'id': 'kruparel@pennmedicine.upenn.edu'},
                 {'access': 'admin', 'id': 'kalebfischer@flywheel.io'},
                 {'access': 'admin', 'id': 'mcieslak@upenn.edu'},
                 {'access': 'admin', 'id': 'kruparel@upenn.edu'},
                 {'access': 'admin', 'id': 'roalf@upenn.edu'},
        

This subject now has a session added to it, whose label is that of the duplicate subject's session

In [18]:
true_example = true_example.reload()

In [22]:
added_ses = true_example.sessions.find_first('label="008454"')
added_ses

{'age': None,
 'analyses': None,
 'created': datetime.datetime(2019, 8, 7, 20, 19, 4, 285000, tzinfo=tzutc()),
 'files': [],
 'group': 'bbl',
 'id': '5d4b323ac3758a0047d862ca',
 'info': {},
 'info_exists': False,
 'label': '008454',
 'modified': datetime.datetime(2019, 8, 7, 20, 19, 4, 285000, tzinfo=tzutc()),
 'notes': [],
 'operator': None,
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'bbl',
             'project': '5c76eee2ba2580003839f897',
             'session': None,
             'subject': '5c7713ccba2580002738df72'},
 'permissions': [{'access': 'admin', 'id': 'ttapera@upenn.edu'}],
 'project': '5c76eee2ba2580003839f897',
 'project_has_template': None,
 'public': None,
 'satisfies_template': None,
 'subject': {'age': None,
             'analyses': None,
             'code': '80263',
             'cohort': None,
             'created': datetime.datetime(2019, 2, 27, 22, 48, 44, 930000, tzinfo=tzutc()),
             'ethnicity': None,
   

Let's add everything from the duplicate session to the new session:

In [24]:
dupe_sessions[0].to_dict()

{'operator': 'POD',
 'public': None,
 'label': '008454',
 'info': {},
 'project': '5c76eee2ba2580003839f897',
 'uid': None,
 'timestamp': datetime.datetime(2013, 8, 30, 10, 18, 14, tzinfo=tzutc()),
 'timezone': None,
 'subject': {'id': '5d306dde35e01a004ad61ee1',
  'project': '5c76eee2ba2580003839f897',
  'public': None,
  'firstname': None,
  'lastname': None,
  'age': None,
  'sex': None,
  'cohort': None,
  'type': None,
  'race': None,
  'ethnicity': None,
  'species': None,
  'strain': None,
  'label': '080263',
  'code': '080263',
  'master_code': None,
  'tags': None,
  'info': {},
  'files': None,
  'parents': {'analysis': None,
   'acquisition': None,
   'session': None,
   'subject': None,
   'project': '5c76eee2ba2580003839f897',
   'group': 'bbl'},
  'created': datetime.datetime(2019, 7, 18, 13, 2, 22, 888000, tzinfo=tzutc()),
  'modified': datetime.datetime(2019, 8, 7, 19, 37, 36, 699000, tzinfo=tzutc()),
  'permissions': [{'id': 'harshakethineni@flywheel.io', 'access': 'a

In [29]:
for k, v in dupe_sessions[0].to_dict().items():
    try:
        added_ses.update({k: v})
        print(k)
    except:
        print("failed for " + k)

operator
failed for public
label
info
project
failed for uid
timestamp
failed for timezone
failed for subject
age
weight
failed for id
failed for info_exists
failed for parents
failed for created
failed for modified
failed for permissions
failed for group
failed for project_has_template
failed for satisfies_template
failed for files
failed for notes
failed for tags
failed for analyses


The above fields can't be added automatically. But this is not a problem except for files. Let's add these now:

In [30]:
added_ses = added_ses.reload()

In [41]:
dupe_ses = dupe_sessions[0]
dupe_ses.download_file(dupe_ses.files[0].name, './{}'.format(dupe_ses.files[0].name))
added_ses.upload_file('./{}'.format(dupe_ses.files[0].name))

This seems the only way to move files from one object to another, by downloading to disk and uploading. 

In [110]:
import os

def file_transfer(origin, dest, tmp):
    
    if not os.path.exists(tmp):
        os.makedirs(tmp)
    file_list = origin.files
    
    print("Downloading from origin...")
    for f in file_list:
        #print("origin.download_file({0}, {1}/{0})".format(f, tmp))
        origin.download_file(f.name, tmp+"/"+f.name)
    
    print("Uploading to destination...")
    for f in file_list:
        #print("dest.upload_file({1}/{0})".format(f, tmp))
        dest.upload_file(tmp+"/"+f.name)
        #print("os.remove({1}/{0})".format(f, tmp))
        os.remove(tmp+"/"+f.name)
        
        # update general
        for k, v in f.to_dict().items():
            try:
                dest.update({k: v})
                #print(k)
            except:
                #print("Skipping general update " + k)
                continue
                
        # update classification
        for k, v in f.classification.items():
            try:
                dest.update_classification({k: v})
                #print(k)
            except:
                # print("Skipping classification update " + k)
                continue
    os.rmdir(tmp)


In [79]:
#file_transfer(dupe_ses, added_ses, "./temp")

Uploading to destination...
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_count
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_co

Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_count
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_count
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update 

Now we need to figure out how to transfer acquisitions

In [81]:
acq = fw.get(dupe_ses.acquisitions()[0].id)

In [82]:
acq

{'analyses': [],
 'collections': None,
 'created': datetime.datetime(2019, 3, 1, 16, 24, 59, 600000, tzinfo=tzutc()),
 'files': [{'classification': {},
            'created': datetime.datetime(2019, 3, 1, 17, 4, 3, 226000, tzinfo=tzutc()),
            'hash': '',
            'id': 'bf83c5b6-43ab-4bd7-a148-60b2b1d5f32e',
            'info': {},
            'info_exists': None,
            'mimetype': 'text/xml',
            'modality': None,
            'modified': datetime.datetime(2019, 3, 1, 17, 4, 2, 889000, tzinfo=tzutc()),
            'name': 'STIM_catalog.xml',
            'origin': {'id': 'harshakethineni@flywheel.io',
                       'method': None,
                       'name': None,
                       'type': 'user',
                       'via': None},
            'replaced': None,
            'size': 564,
            'tags': [],
            'type': 'markup',
            'zip_member_count': None},
           {'classification': {},
            'created': datetime.

In [86]:
#added_acq = added_ses.add_acquisition(label=acq.label)
added_acq = added_acq.reload()
added_acq

{'analyses': [],
 'collections': None,
 'created': datetime.datetime(2019, 8, 8, 20, 29, 9, 577000, tzinfo=tzutc()),
 'files': [],
 'id': '5d4c8615c3758a0049dae234',
 'info': {},
 'info_exists': None,
 'label': '1',
 'modified': datetime.datetime(2019, 8, 8, 20, 29, 9, 577000, tzinfo=tzutc()),
 'notes': [],
 'parents': {'acquisition': None,
             'analysis': None,
             'group': 'bbl',
             'project': '5c76eee2ba2580003839f897',
             'session': '5d4b323ac3758a0047d862ca',
             'subject': '5c7713ccba2580002738df72'},
 'permissions': [{'access': 'admin', 'id': 'harshakethineni@flywheel.io'},
                 {'access': 'admin', 'id': 'kruparel@pennmedicine.upenn.edu'},
                 {'access': 'admin', 'id': 'kalebfischer@flywheel.io'},
                 {'access': 'admin', 'id': 'mcieslak@upenn.edu'},
                 {'access': 'admin', 'id': 'kruparel@upenn.edu'},
                 {'access': 'admin', 'id': 'roalf@upenn.edu'},
                 {'

In [87]:
for k, v in acq.to_dict().items():
    try:
#        added_acq.update({k: v})
        print(k)
    except:
        print("failed for " + k)

failed for public
label
info
session
failed for uid
timestamp
failed for timezone
failed for id
failed for parents
failed for collections
failed for files
failed for created
failed for modified
failed for info_exists
failed for permissions
failed for notes
failed for tags
failed for analyses


In [91]:
#file_transfer(acq, added_acq, 'temp')

Uploading to destination...
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_count
Skipping general update id
Skipping general update name
Skipping general update type
Skipping general update mimetype
Skipping general update modality
Skipping general update classification
Skipping general update tags
info
Skipping general update origin
Skipping general update hash
Skipping general update created
Skipping general update modified
Skipping general update replaced
Skipping general update size
Skipping general update info_exists
Skipping general update zip_member_co

These methods seem to work, so we can go ahead and loop:

In [102]:
dupes

{'080263': '80263',
 '080557': '80557',
 '081222': '81222',
 '081323': '81323',
 '081462': '81462',
 '081939': '81939',
 '082051': '82051',
 '082063': '82063',
 '082066': '82066',
 '083010': '83010',
 '083454': '83454',
 '083835': '83835',
 '083987': '83987',
 '084103': '84103',
 '085063': '85063',
 '085083': '85083',
 '085369': '85369',
 '085400': '85400',
 '085412': '85412',
 '086626': '86626',
 '086946': '86946',
 '087135': '87135',
 '087144': '87144',
 '087166': '87166',
 '087277': '87277',
 '087804': '87804',
 '087990': '87990',
 '088181': '88181',
 '088402': '88402',
 '088589': '88589',
 '088760': '88760',
 '088770': '88770',
 '088773': '88773',
 '088867': '88867',
 '088902': '88902',
 '089115': '89115',
 '089377': '89377',
 '090077': '90077',
 '090113': '90113',
 '090281': '90281',
 '090369': '90369',
 '090613': '90613',
 '090877': '90877',
 '091025': '91025',
 '091335': '91335',
 '091341': '91341',
 '091726': '91726',
 '091855': '91855',
 '091919': '91919',
 '091962': '91962',


In [122]:
#for k, v in dupes.items():
k = '080263'
v = '80263'

print("Copying {0} to {1}...".format(k, v))
origin_sub = [s for s in subs_lg if s.label == k][0]
origin_sub = fw.get(origin_sub.id)

destination_sub = [s for s in subs_lg if s.label == v][0]
destination_sub = fw.get(destination_sub.id)

for ses in origin_sub.sessions():

    # copy the session object
    print("Copying session...")
    origin_ses = fw.get(ses.id)
    destination_ses = destination_sub.add_session(label = origin_ses.label)
    destination_ses = destination_ses.reload()
    
    # copy session metadata
    print("Copying metadata...")
    for meta, val in origin_ses.to_dict().items():
        try:
            destination_ses.update({meta: val})
        except:
            continue

    # copy session files
    print("Transferring files...")
    file_transfer(origin_ses, destination_ses, "temp")

    # get origin acquisitions
    origin_acquisitions = origin_ses.acquisitions()

    for acq in origin_acquisitions:

        # copy the new acquistion object
        print("Copying acquisition {0}...".format(acq.label))
        origin_acq = fw.get(acq.id)
        destination_acq = destination_ses.add_acquisition(label = origin_acq.label)
        destination_acq = destination_acq.reload()

        # copy metadata
#         print("Copying metadata...")
#         for meta, val in origin_acq.to_dict().items():
#             try:
#                 destination_acq.update({meta: val})
#             except:
#                 continue

        #copy acquisition files
        print("Transferring files...")
        file_transfer(origin_acq, destination_acq, "temp")
            
            

Copying 080263 to 80263...
Copying session...
Copying metadata...
Transferring files...
Uploading to destination...
Copying acquisition 1...
Transferring files...
Uploading to destination...
Copying acquisition 1...
Transferring files...
Uploading to destination...
Copying acquisition 10...
Transferring files...
Uploading to destination...
Copying acquisition 10...
Transferring files...
Uploading to destination...
Copying acquisition 11...
Transferring files...
Uploading to destination...
Copying acquisition 11...
Transferring files...
Uploading to destination...
Copying acquisition 12...
Transferring files...
Uploading to destination...
Copying acquisition 12...
Transferring files...
Uploading to destination...
Copying acquisition 2...
Transferring files...
Uploading to destination...
Copying acquisition 2...
Transferring files...
Uploading to destination...
Copying acquisition 3...
Transferring files...
Uploading to destination...
Copying acquisition 3...
Transferring files...
Upload

In [128]:
def copy_subject_data(project, origin, destination):

    print("Working: Subject {}".format(destination))
    print()
    origin_sub = project.subjects.find('label="{}"'.format(origin))[0]
    destination_sub = project.subjects.find('label="{}"'.format(destination))[0]
    
    for ses in origin_sub.sessions():

        # copy the session object
        print("Copying session...")
        origin_ses = fw.get(ses.id)
        destination_ses = destination_sub.add_session(label = origin_ses.label)
        destination_ses = destination_ses.reload()

        # copy session metadata
        print("Copying metadata...")
        for meta, val in origin_ses.to_dict().items():
            try:
                destination_ses.update({meta: val})
            except:
                continue

        # copy session files
        print("Transferring files...")
        file_transfer(origin_ses, destination_ses, "temp")

        # get origin acquisitions
        origin_acquisitions = origin_ses.acquisitions()

        for acq in origin_acquisitions:

            # copy the new acquistion object
            print()
            print("Copying acquisition {0}...".format(acq.label))
            origin_acq = fw.get(acq.id)
            destination_acq = destination_ses.add_acquisition(label = origin_acq.label)
            destination_acq = destination_acq.reload()

            # copy metadata
    #         print("Copying metadata...")
    #         for meta, val in origin_acq.to_dict().items():
    #             try:
    #                 destination_acq.update({meta: val})
    #             except:
    #                 continue

            #copy acquisition files
            print("Transferring files...")
            file_transfer(origin_acq, destination_acq, "temp")

In [129]:
for k, v in dupes.items():

    copy_subject_data(lg, k, v)

Working: Subject 80263

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files.

Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 81939

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 

Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 83454

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 122...
Transferring files...
Uploading to destination...

Copying acquisition 13...
Transferring files...
Uploading to destination...

Copying acquisition 14...
Transferring files...
Uploading to destination...

Copying acquisition 15...
Transferring files...
Uploading to destination...

Copying acquisition 16...
Transferring files...
Uploading to destination...

Copying acquisition 17...
Transferring files...
Uploading to destination...

Copying acqu


Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 13...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working:

Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 87135

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 

...

Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 93274

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 13...
Transferring files...
Uploading to destination...

Copying acquisition 14...
Transferring files...
Uploading to destination...

Copying acquisition 15...
Transferring files...
Uploading to destination...

Copying acquisition 16...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploadi

Transferring files...
Uploading to destination...

Copying acquisition 14...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 95754

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...


Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 13...
Transferring files...
Uploading to destination...

Copying acquisition 2...
Transferring files...
Uploading to destination...

Copying acquisition 3...
Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 97485

Copying session...
Copying metad

Transferring files...
Uploading to destination...

Copying acquisition 4...
Transferring files...
Uploading to destination...

Copying acquisition 5...
Transferring files...
Uploading to destination...

Copying acquisition 6...
Transferring files...
Uploading to destination...

Copying acquisition 7...
Transferring files...
Uploading to destination...

Copying acquisition 8...
Transferring files...
Uploading to destination...

Copying acquisition 9...
Transferring files...
Uploading to destination...
Working: Subject 98831

Copying session...
Copying metadata...
Transferring files...
Uploading to destination...

Copying acquisition 1...
Transferring files...
Uploading to destination...

Copying acquisition 10...
Transferring files...
Uploading to destination...

Copying acquisition 11...
Transferring files...
Uploading to destination...

Copying acquisition 12...
Transferring files...
Uploading to destination...

Copying acquisition 13...
Transferring files...
Uploading to destination.

This seems to have worked for the most part, but some NIFTIs' metadata were not copied over successfully (that's what I get for doing silent try-except blocks). I'll go through each and delete the NIFTI, then use dcm2niix to produce clean new NIFTIs for each.

In [130]:
dcm2nif=fw.get_gear("5cb92ec53767f900271bbc92")

In [142]:
#for k, v in dupes.items():
k = '080263'
v = '80263'   
origin_sub = lg.subjects.find('label="{}"'.format(k))[0]
origin_ses = origin_sub.sessions()[0].label
destination_sub = lg.subjects.find('label="{}"'.format(v))[0]

destination_ses = [fw.get(x.id) for x in destination_sub.sessions() if x.label == origin_ses][0]

acquisitions = [fw.get(x.id) for x in destination_ses.acquisitions()]
for acq in acquisitions:
    print([x.name for x in acq.files if '.nii' in x.name])
    print([x.name for x in acq.files if 'dicom' in x.name or 'dcm' in x.name])

['localizer_1_i00003.nii.gz', 'localizer_1_i00002.nii.gz', 'localizer_1_i00001.nii.gz']
['1.dicom.zip']
['localizer_1_i00003.nii.gz', 'localizer_1_i00002.nii.gz', 'localizer_1_i00001.nii.gz']
['1.dicom.zip']
['DTI_2x32_35_10.nii.gz']
['10.dicom.zip']
['DTI_2x32_35_10.nii.gz']
['10.dicom.zip']
['DTI_2x32_36_11.nii.gz']
['11.dicom.zip']
['DTI_2x32_36_11.nii.gz']
['11.dicom.zip']
['bbl1_restbold1_124_12.nii.gz']
['12.dicom.zip']
['bbl1_restbold1_124_12.nii.gz']
['12.dicom.zip']
['MPRAGE_TI1100_ipat2_2.nii.gz']
['2.dicom.zip']
['MPRAGE_TI1100_ipat2_2.nii.gz']
['2.dicom.zip']
['ep2d_se_pcasl_PHC_1200ms_3.nii.gz']
['3.dicom.zip']
['ep2d_se_pcasl_PHC_1200ms_3.nii.gz']
['3.dicom.zip']
['ep2d_se_pcasl_PHC_1200ms_4.nii.gz', 'ep2d_se_pcasl_PHC_1200ms_MoCo_4.nii.gz']
['4.dicom.zip']
['ep2d_se_pcasl_PHC_1200ms_4.nii.gz', 'ep2d_se_pcasl_PHC_1200ms_MoCo_4.nii.gz']
['4.dicom.zip']
['epi_singlerep_advshim_5.nii.gz']
['5.dicom.zip']
['epi_singlerep_advshim_5.nii.gz']
['5.dicom.zip']
['B0map_onesizefitsa

In [147]:
acq = acquisitions[0]
acq = acq.reload()
niftis = [x.name for x in acq.files if '.nii' in x.name]
print(niftis)
[acq.delete_file(n) for n in niftis]

['localizer_1_i00002.nii.gz', 'localizer_1_i00001.nii.gz']


[{'modified': 1}, {'modified': 1}]

In [150]:
dicom = [x.name for x in acq.files if 'dicom' in x.name or 'dcm' in x.name][0]
inputs = list(dcm2nif.gear.inputs.keys())
gear_input = {inputs[0]: acq.get_file(dicom)}
testJobID = dcm2nif.run(inputs = gear_input, destination = acq)
testJobID

'5d4d9917c3758a0041d63ccc'

In [151]:
#for k, v in dupes.items():
k = '080263'
v = '80263'   
origin_sub = lg.subjects.find('label="{}"'.format(k))[0]
origin_ses = origin_sub.sessions()[0].label
destination_sub = lg.subjects.find('label="{}"'.format(v))[0]

destination_ses = [fw.get(x.id) for x in destination_sub.sessions() if x.label == origin_ses][0]

acquisitions = [fw.get(x.id) for x in destination_ses.acquisitions()]
for acq in acquisitions:
    print("Acquisition ", acq.label)
    print('Deleting niftis...')
    acq = acq.reload()
    niftis = [x.name for x in acq.files if '.nii' in x.name]
    for n in niftis:
        acq.delete_file(n)
        
    print("Replaceing nifti with dcm2niix...")
    dicom = [x.name for x in acq.files if 'dicom' in x.name or 'dcm' in x.name][0]
    inputs = list(dcm2nif.gear.inputs.keys())
    gear_input = {inputs[0]: acq.get_file(dicom)}
    jobID = dcm2nif.run(inputs = gear_input, destination = acq)
    print(jobID)

Acquisition  1
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab2c3758a0041d63dcc
Acquisition  1
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab3c3758a0045d801e5
Acquisition  10
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab3c3758a006ed6ea7c
Acquisition  10
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab3c3758a006ed6ea7e
Acquisition  11
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab4c3758a0045d801e9
Acquisition  11
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab4c3758a0049dbb9aa
Acquisition  12
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab4c3758a003ed57255
Acquisition  12
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab4c3758a006ed6ea81
Acquisition  2
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab5c3758a006ed6ea82
Acquisition  2
Deleting niftis...
Replaceing nifti with dcm2niix...
5d4d9ab5c3758a006ed6ea83
Acquisition  3
Deleting niftis...
Replaceing nifti with dcm2niix

In [153]:
for k, v in dupes.items():
    print("Processing subject ", v)
    origin_sub = lg.subjects.find('label="{}"'.format(k))[0]
    origin_ses = origin_sub.sessions()[0].label
    destination_sub = lg.subjects.find('label="{}"'.format(v))[0]

    destination_ses = [fw.get(x.id) for x in destination_sub.sessions() if x.label == origin_ses][0]

    acquisitions = [fw.get(x.id) for x in destination_ses.acquisitions()]
    for acq in acquisitions:
        print("Acquisition ", acq.label)
        acq = acq.reload()
        niftis = [x.name for x in acq.files if '.nii' in x.name or '.bval' in x.name or '.bvec' in x.name]
        if niftis:
            for n in niftis:
                acq.delete_file(n)

        dicom = [x.name for x in acq.files if 'dicom' in x.name or 'dcm' in x.name]
        if dicom:
            inputs = list(dcm2nif.gear.inputs.keys())
            gear_input = {inputs[0]: acq.get_file(dicom[0])}
            jobID = dcm2nif.run(inputs = gear_input, destination = acq)
            print(jobID)
    print()

Processing subject  80263
Acquisition  1
5d4d9cc9c3758a0043d6d46e
Acquisition  1
5d4d9cc9c3758a003bd50238
Acquisition  10
5d4d9ccac3758a0049dbbb62
Acquisition  10
5d4d9ccac3758a0043d6d473
Acquisition  11
5d4d9ccbc3758a0045d80333
Acquisition  11
5d4d9ccbc3758a0041d63ed6
Acquisition  12
5d4d9cccc3758a006ed6ec6d
Acquisition  12
5d4d9cccc3758a003ed57345
Acquisition  2
5d4d9cccc3758a0035d48710
Acquisition  2
5d4d9cccc3758a0045d80336
Acquisition  3
5d4d9cccc3758a0047d9ff53
Acquisition  3
5d4d9ccdc3758a0045d80337
Acquisition  4
5d4d9ccdc3758a006ed6ec75
Acquisition  4
5d4d9ccdc3758a0047d9ff57
Acquisition  5
5d4d9ccec3758a0047d9ff59
Acquisition  5
5d4d9ccec3758a002dd44370
Acquisition  6
5d4d9ccec3758a0049dbbb6f
Acquisition  6
5d4d9ccec3758a0047d9ff5e
Acquisition  7
5d4d9ccfc3758a0047d9ff60
Acquisition  7
5d4d9ccfc3758a003ed5734b
Acquisition  8
5d4d9ccfc3758a006ed6ec7f
Acquisition  8
5d4d9cd0c3758a006ed6ec81
Acquisition  9
5d4d9cd0c3758a0035d48714
Acquisition  9
5d4d9cd0c3758a0047d9ff62
Acquisit

Acquisition  1
5d4d9d5bc3758a0041d63f61
Acquisition  2
5d4d9d5cc3758a0043d6d50c
Acquisition  3
5d4d9d5cc3758a006ed6ed5f
Acquisition  4
5d4d9d5cc3758a0041d63f65
Acquisition  5
5d4d9d5dc3758a0033d45f70
Acquisition  6
5d4d9d5dc3758a002fd448ad
Acquisition  7
5d4d9d5ec3758a0045d80405
Acquisition  8
5d4d9d5ec3758a0045d80408
Acquisition  9
5d4d9d5fc3758a0031d453e8

Processing subject  85063
Acquisition  1
5d4d9d65c3758a0047da001c
Acquisition  10
5d4d9d66c3758a0047da001e
Acquisition  11
5d4d9d67c3758a0047da0025
Acquisition  12
5d4d9d69c3758a0043d6d522
Acquisition  13
5d4d9d6ac3758a0043d6d524
Acquisition  2
5d4d9d6ac3758a003bd502a2
Acquisition  3
5d4d9d6bc3758a0041d63f73
Acquisition  4
5d4d9d6bc3758a003bd502a4
Acquisition  5
5d4d9d6bc3758a0047da0031
Acquisition  6
5d4d9d6cc3758a0043d6d527
Acquisition  7
5d4d9d6cc3758a0037d4be94
Acquisition  8
5d4d9d6dc3758a003ed573c2
Acquisition  9
5d4d9d6dc3758a002dd44391

Processing subject  85083
Acquisition  1
5d4d9d71c3758a0047da003a
Acquisition  10
5d4d9d

5d4d9e08c3758a0047da00fe
Acquisition  6
5d4d9e08c3758a003ed57451
Acquisition  7
5d4d9e09c3758a0047da0101
Acquisition  8
5d4d9e09c3758a0047da0102
Acquisition  9
5d4d9e09c3758a0049dbbcda

Processing subject  88402
Acquisition  1
5d4d9e0dc3758a003bd502fb
Acquisition  10
5d4d9e0dc3758a0031d4541c
Acquisition  11
5d4d9e0ec3758a0041d64014
Acquisition  114
Acquisition  12
5d4d9e0ec3758a0043d6d5a0
Acquisition  13
5d4d9e0fc3758a0041d64015
Acquisition  133
Acquisition  14
5d4d9e10c3758a003ed57459
Acquisition  15
5d4d9e10c3758a002fd448ed
Acquisition  16
5d4d9e11c3758a002dd443c2
Acquisition  17
5d4d9e11c3758a0035d487f1
Acquisition  18
5d4d9e13c3758a0041d64021
Acquisition  19
5d4d9e14c3758a0033d45fb7
Acquisition  2
5d4d9e18c3758a0033d45fba
Acquisition  20
5d4d9e19c3758a0043d6d5b9
Acquisition  3
5d4d9e19c3758a003bd5030a
Acquisition  4
5d4d9e19c3758a0033d45fbb
Acquisition  5
5d4d9e1ec3758a003ed5745e
Acquisition  6
5d4d9e1ec3758a0047da0123
Acquisition  7
5d4d9e1fc3758a003bd5030d
Acquisition  8
5d4d9e1f

5d4d9ec3c3758a006ed6ef6c
Acquisition  11
5d4d9ec4c3758a006ed6ef6e
Acquisition  12
5d4d9ec4c3758a002dd443fd
Acquisition  13
5d4d9ec5c3758a006ed6ef72
Acquisition  137
Acquisition  14
5d4d9ec6c3758a0031d45463
Acquisition  15
5d4d9ec6c3758a0041d640c2
Acquisition  2
5d4d9ec7c3758a0043d6d675
Acquisition  3
5d4d9ec9c3758a0041d640c3
Acquisition  4
5d4d9ecac3758a0049dbbdcb
Acquisition  5
5d4d9ecac3758a0037d4bf6e
Acquisition  6
5d4d9ecbc3758a0037d4bf6f
Acquisition  7
5d4d9ecbc3758a006ed6ef7a
Acquisition  8
5d4d9ecbc3758a006ed6ef7b
Acquisition  9
5d4d9eccc3758a0041d640c6

Processing subject  90877
Acquisition  1
5d4d9ecec3758a0043d6d67b
Acquisition  10
5d4d9ecec3758a0049dbbdd1
Acquisition  11
5d4d9ecfc3758a0049dbbdd3
Acquisition  12
5d4d9ecfc3758a0033d46000
Acquisition  13
5d4d9ed0c3758a006ed6ef81
Acquisition  2
5d4d9ed1c3758a0045d8058c
Acquisition  3
5d4d9ed1c3758a0045d8058d
Acquisition  4
5d4d9ed1c3758a006ed6ef88
Acquisition  5
5d4d9ed2c3758a0045d8058f
Acquisition  6
5d4d9ed2c3758a0035d4883c
Ac

5d4d9f5dc3758a0045d80600
Acquisition  5
5d4d9f5dc3758a0049dbbe5f
Acquisition  6
5d4d9f5dc3758a0041d64126
Acquisition  7
5d4d9f5ec3758a006ed6f029
Acquisition  8
5d4d9f5ec3758a0031d4548e
Acquisition  9
5d4d9f5ec3758a006ed6f02b

Processing subject  93242
Acquisition  1
5d4d9f61c3758a0047da025e
Acquisition  10
5d4d9f62c3758a0045d80604
Acquisition  11
5d4d9f63c3758a0047da0262
Acquisition  12
5d4d9f63c3758a0047da0264
Acquisition  13
5d4d9f63c3758a006ed6f032
Acquisition  2
5d4d9f64c3758a0045d80607
Acquisition  3
5d4d9f64c3758a003dd55768
Acquisition  4
5d4d9f65c3758a0049dbbe61
Acquisition  5
5d4d9f65c3758a002fd44951
Acquisition  6
5d4d9f66c3758a0043d6d6cf
Acquisition  7
5d4d9f69c3758a006ed6f035
Acquisition  8
5d4d9f6ac3758a0043d6d6d1
Acquisition  9
5d4d9f6cc3758a0033d4602a

Processing subject  93274
Acquisition  1
5d4d9f6ec3758a0043d6d6d6
Acquisition  10
5d4d9f6fc3758a006ed6f03f
Acquisition  11
5d4d9f6fc3758a0049dbbe6d
Acquisition  2
5d4d9f6fc3758a0049dbbe6e
Acquisition  3
5d4d9f70c3758a003dd5

5d4da026c3758a0049dbbf75
Acquisition  2
5d4da026c3758a0027d4403e
Acquisition  3
5d4da027c3758a002fd449a2
Acquisition  4
5d4da027c3758a0031d454e3
Acquisition  5
5d4da028c3758a003ed575dd
Acquisition  6
5d4da028c3758a0049dbbf7d
Acquisition  7
5d4da029c3758a0045d806d3
Acquisition  8
5d4da029c3758a0033d4607d
Acquisition  9
5d4da029c3758a0047da0345

Processing subject  96420
Acquisition  1
5d4da030c3758a0037d4c046
Acquisition  10
5d4da031c3758a002bd43c3f
Acquisition  11
5d4da031c3758a0037d4c04a
Acquisition  12
5d4da032c3758a003bd5043a
Acquisition  13
5d4da033c3758a006ed6f143
Acquisition  2
5d4da033c3758a0031d454e7
Acquisition  3
5d4da034c3758a0033d46085
Acquisition  4
5d4da034c3758a0047da0347
Acquisition  5
5d4da037c3758a0035d488dc
Acquisition  6
5d4da03ac3758a003ed575ff
Acquisition  7
5d4da03dc3758a003bd5043e
Acquisition  8
5d4da03dc3758a006ed6f14f
Acquisition  9
5d4da03ec3758a0047da034c

Processing subject  96659
Acquisition  1
5d4da041c3758a003ed57607
Acquisition  10
5d4da042c3758a0041d64

Success!

Now we delete the original duplicates

In [160]:
origin_sub = lg.subjects.find('label="{}"'.format('080263'))[0]
lg.delete_subject('080263')

AttributeError: type object 'object' has no attribute 'delete_subject'

In [ ]:
for k, v in dupes.items():
    origin_sub = lg.subjects.find('label="{}"'.format(k))[0]
    